In [1]:
import qcodes as qc
station = qc.Station(config_file='config/station.yaml')

In [2]:
dac1 = station.load_mock_dac1()
dac2 = station.load_mock_dac2()

In [4]:
%load_ext instrmcp.extensions

The instrmcp.extensions extension is already loaded. To reload it, use:
  %reload_ext instrmcp.extensions


In [3]:
%mcp_status

🛡️ MCP Server Status:
   Desired Mode: safe
   Server Running: ✅
   Host: 127.0.0.1:8123
   Task: ❌ Inactive
   Available: %mcp_close, %mcp_restart


In [6]:
%mcp_option add measureit database auto_correct_json

✅ Added: measureit
✅ Added: database
✅ Added: auto_correct_json
✅ Changes will take effect when server starts


In [4]:
%mcp_unsafe

⚠️  Mode set to unsafe
⚠️  UNSAFE MODE: execute_editing_cell tool will be available
⚠️  Server restart required for tool changes to take effect
   Use: %mcp_restart


In [8]:
%mcp_dangerous

⚠️⚠️⚠️  DANGEROUS MODE ENABLED  ⚠️⚠️⚠️
All consent dialogs will be automatically approved!
This mode bypasses all safety confirmations.
✅ Mode will take effect when server starts


In [9]:
%mcp_start

2025-11-30 15:50:58,931 ¦ instrmcp.servers.jupyter_qcodes.security.consent ¦ WARNING ¦ consent ¦ __init__ ¦ 61 ¦ ⚠️  CONSENT BYPASS MODE ENABLED - All operations auto-approved!
2025-11-30 15:50:58,937 ¦ instrmcp.servers.jupyter_qcodes.security.consent ¦ WARNING ¦ consent ¦ __init__ ¦ 61 ¦ ⚠️  CONSENT BYPASS MODE ENABLED - All operations auto-approved!


🚀 Starting MCP server...
🚀 QCoDeS MCP Server running on http://127.0.0.1:8123
🔑 Access token: YI1Og87_XIP8kZCsJ_EIk-9VP-czZGpDGFYqMwwAjZw
✅ MCP server started in ☠️ dangerous mode
⚠️⚠️⚠️  All consent dialogs auto-approved!


[11/30/25 15:50:58] INFO     Starting MCP server 'Jupyter QCoDeS MCP Server (Unsafe Mode)' with      ]8;id=935844;file:///Users/caijiaqi/miniforge3/envs/instrMCPdev/lib/python3.13/site-packages/fastmcp/server/server.py\server.py]8;;\:]8;id=27122;file:///Users/caijiaqi/miniforge3/envs/instrMCPdev/lib/python3.13/site-packages/fastmcp/server/server.py#2050\2050]8;;\
                             transport 'http' on http://127.0.0.1:8123/mcp                                         

INFO:     Started server process [40429]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 48] error while attempting to bind on address ('127.0.0.1', 8123): [errno 48] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


In [5]:
2222

2222

In [ ]:
asdf